In [1]:
import os
from io import StringIO

import sys

current_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'PVT-thermal-performance-model', 'CFD')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'RD-systems-and-test-benches')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss')))
sys.path.append(os.path.abspath(os.path.join(current_dir, '..', '..', 'parallel-flow-distribution-pressure-loss', 'ansys')))

import numpy as np
import pandas as pd
import openpyxl

import itertools

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import jou_gen as jg
import bridge_module as bm

import ansys.fluent.core as pyfluent

from CoolProp.CoolProp import PropsSI

import hx_hydraulic as hxhy

In [2]:
computer = 'lmps_cds'
jg.set_computer(computer)
server_number = 1104

category_folder_name = 'Conduction'
project_folder_name = 'fin_for_1Dmodel'

caoMeshFolder = 'TS_M1'
title = 'abaque0_try0'

nb_it = 30000

report_names = ['report_total_ht',
                'report_tube_int_ht'
                ]

In [ ]:
if computer == 'seagull':
    root_fp = r'D:\ANSYS Fluent Projects'
else:
    root_fp = "/usrtmp/delachaux/ANSYS Fluent Projects"
    jg.computer = computer

main_fp = os.path.join(root_fp, category_folder_name, project_folder_name)
Mesh_fp = os.path.join(main_fp, 'mesh')
SR_fp = bm.check_folder(os.path.join(main_fp, 'SimulationResults'))
caoMesh_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder))
param_fp = bm.check_folder(os.path.join(SR_fp, caoMeshFolder, title))

tui, solver = bm.init_solver(main_fp, str(server_number), subfolder='')
print('main_fp :',main_fp)
print('SR_fp :',SR_fp)

In [ ]:
# Define the parameter ranges
h_back_tube = [1, 3, 5]
h_back = [1, 2, 3]
T_amb = [267.15, 270.15, 273.15]

# Generate combinations for T_tube and T_PV based on T_amb
combinations = []
for T_amb_value in T_amb:
    T_tube = [T_amb_value - 6, T_amb_value - 9]
    T_PV = [T_amb_value - 2, T_amb_value - 4, T_amb_value - 6]
    
    # Create a cartesian product for the other parameters with T_tube and T_PV
    param_combinations = itertools.product(h_back_tube, h_back, [T_amb_value], T_tube, T_PV)
    combinations.extend(param_combinations)

# Convert combinations into a DataFrame
columns = ['h_back_tube', 'h_back', 'T_amb', 'T_tube', 'T_PV']
parametric_df = pd.DataFrame(combinations, columns=columns)
parametric_df['h_fins'] = parametric_df['h_back_tube']
parametric_df['case'] = 'case' + parametric_df.index.astype(str)

parametric_df

In [ ]:
for index, row in parametric_df.iterrows():

    if index >+ 0:

        case_name = str(row['case'])
        folder_path_case = bm.check_folder(os.path.join(SR_fp, title, case_name))

        for report_name in report_names:
            jg.change_report_file_path(tui, report_name, os.path.join(folder_path_case, f"{report_name}.out"))

        jg.change_named_expression(tui, 'h_back_tube', row['h_back_tube'], 'W/(m^2 K)')
        jg.change_named_expression(tui, 'h_fins', row['h_fins'], 'W/(m^2 K)')
        jg.change_named_expression(tui, 'h_back', row['h_back'], 'W/(m^2 K)')
        jg.change_named_expression(tui, 'T_amb', row['T_amb'], 'K')
        jg.change_named_expression(tui, 'T_tube', row['T_tube'], 'K')
        jg.change_named_expression(tui, 'T_PV', row['T_PV'], 'K')

        solver.solution.initialization.hybrid_initialize()

        if computer == 'seagull':
            solver.solution.run_calculation.iterate(number_of_iterations = nb_it)
        elif computer == 'lmps_cds':
            solver.solution.run_calculation.iterate(iter_count = nb_it)

        jg.write_report(tui, 'ht', folder_path_case, 'ht_report')
        jg.write_residuals_file(tui, folder_path_case, 'residuals')
        jg.write_data(tui, folder_path_case, case_name)